In [1]:
from pyspark.sql.functions import explode, split, trim, lower, expr
from pyspark.sql import SparkSession, DataFrame
from pyspark import SparkConf
from pyspark import SparkContext
import logging
from os.path import abspath
from pathlib import Path
import shutil
from pathlib import Path
from typing import Optional, Union, List, Tuple, Any

In [2]:
print(sc._jvm.org.apache.hadoop.util.VersionInfo.getVersion())

3.3.4


In [3]:
print(spark._jsc.hadoopConfiguration().get("javax.jdo.option.ConnectionURL"))

jdbc:derby:;databaseName=/opt/spark/warehouse/metastore_db;create=true


In [4]:
print(spark._jsc.hadoopConfiguration().get("hive.metastore.warehouse.dir"))

/opt/spark/warehouse/


25/05/20 13:44:22 INFO  StandaloneSchedulerBackend$StandaloneDriverEndpoint:60 Registered executor NettyRpcEndpointRef(spark-client://Executor) (172.18.0.3:52364) with ID 1,  ResourceProfileId 0
25/05/20 13:44:22 INFO  StandaloneSchedulerBackend$StandaloneDriverEndpoint:60 Registered executor NettyRpcEndpointRef(spark-client://Executor) (172.18.0.6:36028) with ID 2,  ResourceProfileId 0
25/05/20 13:44:22 INFO  StandaloneSchedulerBackend$StandaloneDriverEndpoint:60 Registered executor NettyRpcEndpointRef(spark-client://Executor) (172.18.0.4:59772) with ID 0,  ResourceProfileId 0
25/05/20 13:44:22 INFO  BlockManagerMasterEndpoint:60 Registering block manager 172.18.0.3:45939 with 1048.8 MiB RAM, BlockManagerId(1, 172.18.0.3, 45939, None)
25/05/20 13:44:22 INFO  BlockManagerMasterEndpoint:60 Registering block manager 172.18.0.6:37821 with 1048.8 MiB RAM, BlockManagerId(2, 172.18.0.6, 37821, None)
25/05/20 13:44:22 INFO  BlockManagerMasterEndpoint:60 Registering block manager 172.18.0.4:38

In [5]:
print(spark.conf.get("spark.sql.catalogImplementation"))

25/05/20 13:44:38 INFO  SharedState:60 Setting hive.metastore.warehouse.dir ('/opt/spark/warehouse/') to the value of spark.sql.warehouse.dir.
25/05/20 13:44:38 WARN  MetricsConfig:136 Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/05/20 13:44:38 INFO  MetricsSystemImpl:378 Scheduled Metric snapshot period at 10 second(s).
25/05/20 13:44:38 INFO  MetricsSystemImpl:191 s3a-file-system metrics system started
25/05/20 13:44:38 WARN  VersionInfoUtils:85 The AWS SDK for Java 1.x entered maintenance mode starting July 31, 2024 and will reach end of support on December 31, 2025. For more information, see https://aws.amazon.com/blogs/developer/the-aws-sdk-for-java-1-x-is-in-maintenance-mode-effective-july-31-2024/
You can print where on the file system the AWS SDK for Java 1.x core runtime is located by setting the AWS_JAVA_V1_PRINT_LOCATION environment variable or aws.java.v1.printLocation system property to 'true'.
This message can

hive


In [6]:
spark.stop()

25/05/20 13:44:40 INFO  SparkContext:60 SparkContext is stopping with exitCode 0.
25/05/20 13:44:40 INFO  SparkUI:60 Stopped Spark web UI at http://d0f9a55c06b4:4040
25/05/20 13:44:40 INFO  StandaloneSchedulerBackend:60 Shutting down all executors
25/05/20 13:44:40 INFO  StandaloneSchedulerBackend$StandaloneDriverEndpoint:60 Asking each executor to shut down
25/05/20 13:44:40 INFO  MapOutputTrackerMasterEndpoint:60 MapOutputTrackerMasterEndpoint stopped!
25/05/20 13:44:40 INFO  MemoryStore:60 MemoryStore cleared
25/05/20 13:44:40 INFO  BlockManager:60 BlockManager stopped
25/05/20 13:44:40 INFO  BlockManagerMaster:60 BlockManagerMaster stopped
25/05/20 13:44:40 INFO  OutputCommitCoordinator$OutputCommitCoordinatorEndpoint:60 OutputCommitCoordinator stopped!
25/05/20 13:44:40 INFO  SparkContext:60 Successfully stopped SparkContext


In [7]:
spark = (
    SparkSession.builder
    .appName("Init DB")
    .enableHiveSupport()
    .getOrCreate()
)

25/05/20 13:44:41 INFO  SparkContext:60 Running Spark version 3.5.5
25/05/20 13:44:41 INFO  SparkContext:60 OS info Linux, 5.15.167.4-microsoft-standard-WSL2, amd64
25/05/20 13:44:41 INFO  SparkContext:60 Java version 17.0.14
Setting Spark log level to "INFO".
25/05/20 13:44:41 INFO  ResourceUtils:60 ==============================================================
25/05/20 13:44:41 INFO  ResourceUtils:60 No custom resources configured for spark.driver.
25/05/20 13:44:41 INFO  ResourceUtils:60 ==============================================================
25/05/20 13:44:41 INFO  SparkContext:60 Submitted application: Init DB
25/05/20 13:44:41 INFO  ResourceProfile:60 Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 4, script: , vendor: , memory -> name: memory, amount: 2048, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
25/05/20 13:44:41 INFO  ResourceProfile:60 Limit

In [8]:
# Create dw schema in catalog
spark.sql("create database if not exists edw")
spark.sql("create database if not exists edw_stg")
spark.sql("create database if not exists edw_ld")
spark.sql("show databases").show()

25/05/20 13:44:42 INFO  SharedState:60 Setting hive.metastore.warehouse.dir ('/opt/spark/warehouse/') to the value of spark.sql.warehouse.dir.
25/05/20 13:44:42 INFO  SharedState:60 Warehouse path is 's3a://delta-datawarehouse'.
25/05/20 13:44:44 INFO  HiveUtils:60 Initializing HiveMetastoreConnection version 2.3.9 using Spark classes.
25/05/20 13:44:43 WARN  HiveConf:4122 HiveConf of name hive.stats.jdbc.timeout does not exist
25/05/20 13:44:43 WARN  HiveConf:4122 HiveConf of name hive.metastore.schema.version does not exist
25/05/20 13:44:43 WARN  HiveConf:4122 HiveConf of name hive.stats.retries.wait does not exist
25/05/20 13:44:43 INFO  StandaloneSchedulerBackend$StandaloneDriverEndpoint:60 Registered executor NettyRpcEndpointRef(spark-client://Executor) (172.18.0.3:40860) with ID 1,  ResourceProfileId 0
25/05/20 13:44:43 INFO  StandaloneSchedulerBackend$StandaloneDriverEndpoint:60 Registered executor NettyRpcEndpointRef(spark-client://Executor) (172.18.0.6:44940) with ID 2,  Reso

AnalysisException: org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient

In [9]:
# Create Store Dim table
spark.sql("""drop table if exists edw.dim_store""")

spark.sql("""
create table edw.dim_store (
    row_wid string,
    store_id string,
    store_name string,
    address string,
    city string,
    state string,
    zip_code string,
    phone_number string,
    rundate string,
    insert_dt timestamp,
    update_dt timestamp
)
USING delta
""")

print("SPARK-APP: Store dimension created")

25/05/20 11:25:55 INFO  HiveMetaStore:781 0: get_database: global_temp
25/05/20 11:25:55 INFO  audit:309 ugi=root	ip=unknown-ip-addr	cmd=get_database: global_temp	
25/05/20 11:25:55 WARN  ObjectStore:723 Failed to get database global_temp, returning NoSuchObjectException
25/05/20 11:25:55 INFO  HiveMetaStore:781 0: get_database: edw
25/05/20 11:25:55 INFO  audit:309 ugi=root	ip=unknown-ip-addr	cmd=get_database: edw	
25/05/20 11:25:55 INFO  HiveMetaStore:781 0: get_table : db=edw tbl=dim_store
25/05/20 11:25:55 INFO  audit:309 ugi=root	ip=unknown-ip-addr	cmd=get_table : db=edw tbl=dim_store	
25/05/20 11:25:55 INFO  HiveMetaStore:781 0: get_database: edw
25/05/20 11:25:55 INFO  audit:309 ugi=root	ip=unknown-ip-addr	cmd=get_database: edw	
25/05/20 11:25:55 INFO  HiveMetaStore:781 0: get_table : db=edw tbl=dim_store
25/05/20 11:25:55 INFO  audit:309 ugi=root	ip=unknown-ip-addr	cmd=get_table : db=edw tbl=dim_store	


Py4JJavaError: An error occurred while calling o102.sql.
: java.lang.NoClassDefFoundError: io/delta/storage/commit/actions/AbstractProtocol
	at java.base/java.lang.ClassLoader.defineClass1(Native Method)
	at java.base/java.lang.ClassLoader.defineClass(ClassLoader.java:1017)
	at java.base/java.security.SecureClassLoader.defineClass(SecureClassLoader.java:150)
	at java.base/jdk.internal.loader.BuiltinClassLoader.defineClass(BuiltinClassLoader.java:862)
	at java.base/jdk.internal.loader.BuiltinClassLoader.findClassOnClassPathOrNull(BuiltinClassLoader.java:760)
	at java.base/jdk.internal.loader.BuiltinClassLoader.loadClassOrNull(BuiltinClassLoader.java:681)
	at java.base/jdk.internal.loader.BuiltinClassLoader.loadClass(BuiltinClassLoader.java:639)
	at java.base/jdk.internal.loader.ClassLoaders$AppClassLoader.loadClass(ClassLoaders.java:188)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:525)
	at org.apache.spark.sql.delta.actions.Protocol$.apply(actions.scala:205)
	at org.a

In [ ]:
# Create Plan Type Dimension
spark.sql("""drop table if exists edw.dim_plan_type""")
spark.sql("""
create table edw.dim_plan_type (
    plan_type_code string,
    plan_name string,
    rundate string,
    insert_dt timestamp,
    update_dt timestamp
)
USING delta
;
""")

print("SPARK-APP: Plan Type dimension created")

In [ ]:
# Create Date Dimension
spark.sql("""drop table if exists edw.dim_date""")
spark.sql("""
create table edw.dim_date (
    row_wid string,
    date date,
    day int,
    month int,
    year int,
    day_of_week string,
    rundate string,
    insert_dt timestamp,
    update_dt timestamp
)
USING delta
;
""")

print("SPARK-APP: Date dimension created")

In [ ]:
# Create Product Dimension
spark.sql("""drop table if exists edw.dim_product""")
spark.sql("""
create table edw.dim_product (
    row_wid string,
    product_id string,
    product_name string,
    brand string,
    type string,
    flavor string,
    size string,
    price double,
    expiration_dt date,
    image_url string,
    effective_start_dt timestamp,
    effective_end_dt timestamp,
    active_flg int,
    rundate string,
    insert_dt timestamp,
    update_dt timestamp
)
USING delta
;
""")

print("SPARK-APP: Product dimension created")

In [ ]:
# Create Customer Dimension
spark.sql("""drop table if exists edw.dim_customer""")
spark.sql("""
create table edw.dim_customer (
    row_wid string,
    customer_id string,
    first_name string,
    last_name string,
    address string,
    city string,
    state string,
    zip_code string,
    phone_number string,
    email string,
    date_of_birth date,
    plan_type string,
    effective_start_dt timestamp,
    effective_end_dt timestamp,
    active_flg int,
    rundate string,
    insert_dt timestamp,
    update_dt timestamp
)
USING delta
;
""")

print("SPARK-APP: Customer dimension created")

In [ ]:
# Create Sales Fact
spark.sql("""drop table if exists edw.fact_sales""")
spark.sql("""
create table edw.fact_sales (
    date_wid string,
    product_wid string,
    store_wid string,
    customer_wid string,
    order_id string,
    invoice_num string,
    qty int,
    tax double,
    discount double,
    line_total double,
    integration_key string,
    rundate string,
    insert_dt timestamp,
    update_dt timestamp
)
USING delta
;
""")

print("SPARK-APP: Sales Fact created")

In [ ]:
# Create Audit table
spark.sql("""drop table if exists edw.job_control""")
spark.sql("""
create table edw.job_control (
    schema_name string,
    table_name string,
    max_timestamp timestamp,
    rundate string,
    insert_dt timestamp
)
USING delta
;
""")

print("SPARK-APP: JOB Control table created")

In [ ]:
# Log all tables in Data Warehouse/Lakehouse

spark.sql("show tables in edw").show()

In [ ]:
spark.stop()